In [1]:
# Initial imports
import pandas as pd
import alpaca_trade_api as tradeapi
from finta import TA
import numpy as np


In [2]:
# API credentials
ALPACA_API_KEY = 'PKETI0Q5C8PPUXVNHJFJ'
ALPACA_SECRET_KEY = 'WQxmEpBCoMXydCCg0G8cUe6hGLpgaAfrvedZ09Fy'
ALPACA_API_BASE_URL = "https://paper-api.alpaca.markets"

# Create a connection to the API 
api = tradeapi.REST(ALPACA_API_KEY, ALPACA_SECRET_KEY, ALPACA_API_BASE_URL, api_version="v2")

# Set signal variable
signal = 1

# Create buy signal, num shares and ticker
if signal == 1:
    orderSide = "buy"
else:
    orderSide = "sell"
    
# Set the ticket symbol and the number of shares to buy
ticker = "AAPL"
number_of_shares = 1

# Make API call
prices = api.get_bars(ticker, "15Min", "2022-08-22", "2022-10-14", adjustment='raw').df
# Reorganize the DataFrame
prices = pd.concat([prices], axis=1, keys=["TSLA"])

# Drop the Multi-Index from the DataFrame
prices.columns = prices.columns.droplevel(0)
# create a seperate dataframe for signals
signals_df = prices
signals_df

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2022-08-22 05:00:00+00:00,169.65,169.80,169.65,169.80,521,6,169.756526
2022-08-22 06:00:00+00:00,169.22,169.22,169.22,169.22,100,1,169.220000
2022-08-22 06:45:00+00:00,169.20,169.20,169.20,169.20,299,1,169.200000
2022-08-22 07:15:00+00:00,169.10,169.10,169.10,169.10,114,6,169.101754
2022-08-22 08:00:00+00:00,170.29,170.29,169.01,169.29,69053,1471,169.268029
...,...,...,...,...,...,...,...
2022-10-14 22:45:00+00:00,138.14,138.14,138.04,138.07,11808,274,138.074685
2022-10-14 23:00:00+00:00,138.07,138.07,138.00,138.04,9133,379,138.016969
2022-10-14 23:15:00+00:00,138.04,138.08,138.02,138.05,8187,203,138.043907


In [3]:
#  Setup EMAs for crosses
signals_df["9EMA"] = TA.EMA(signals_df, 9)
signals_df["20EMA"] = TA.EMA(signals_df, 20)
signals_df["50EMA"] = TA.EMA(signals_df, 50)
signals_df["200SMA"] = TA.SMA(signals_df, 200)

# Setup Indicators
signals_df["ATR"] = TA.ATR(signals_df)
bbands_df = TA.BBANDS(signals_df)
macd_df = TA.MACD(signals_df)
signals_df["OBV"] = TA.OBV(signals_df)
signals_df["RSI"] = TA.RSI(signals_df)

# join macd and bbands Dataframes to signals_df
bbands_df = pd.concat([bbands_df, macd_df], axis=1)
signals_df = pd.concat([signals_df, bbands_df], axis=1)
signals_df.drop(columns="SIGNAL", inplace=True)
signals_df

# Position Relative to BollingerBands, Above Upper_BB = 1, Below Lower_BB = -1, Inbetween both = 0
signals_df["Bollinger_Bands_Above_Upper_BB"] = 0.0
# 9EMA is above 20EMA = 1
signals_df["9EMA/20EMA_Cross, 9EMA>20EMA"] = 0.0
# 50 EMA is above 200SMA = 1
signals_df["50EMA/200SMA_Cross, 50EMA>200SMA"] = 0.0
# MACD Increasing = 1, Decreasing = -1
# signals_df["MACD_Rate"] = 0.0
# OBV is positive = 1, OBV is 
signals_df["OBV_Zero_Cross_From_Above"] = 0.0
# If RSI is above 70: +1, If RSI below 30: -1
signals_df["RSI_Over_70"] = 0.0
# Price is crossing above VWAP = 1
#signals_df["VWAP_Cross_From_Above"] = 0.0

# Position Relative to BollingerBands, Above Upper_BB = 1, Below Lower_BB = -1, Inbetween both = 0
signals_df["Bollinger_Bands_Below_Lower_BB"] = 0.0
# 9EMA is above 20EMA = 1
signals_df["9EMA/20EMA_Cross, 9EMA<20EMA"] = 0.0
# 50 EMA is above 200SMA = 1
signals_df["50EMA/200SMA_Cross, 50EMA<200SMA"] = 0.0
# MACD Increasing = 1, Decreasing = -1
# signals_df["MACD_Rate"] = 0.0
# OBV is positive = 1, OBV is 
signals_df["OBV_Zero_Cross_From_Below"] = 0.0
# If RSI is above 70: +1, If RSI below 30: -1
signals_df["RSI_Under_30"] = 0.0
# Price is crossing above VWAP = 1
signals_df["VWAP_Cross_From_Above"] = 0.0
# Price is crossing above VWAP = 1
signals_df["VWAP_Cross_From_Below"] = 0.0

# Review DataFrame
signals_df.head()

,open,high,low,close,volume,trade_count,vwap,9EMA,20EMA,50EMA,...,"50EMA/200SMA_Cross, 50EMA>200SMA",OBV_Zero_Cross_From_Above,RSI_Over_70,Bollinger_Bands_Below_Lower_BB,"9EMA/20EMA_Cross, 9EMA<20EMA","50EMA/200SMA_Cross, 50EMA<200SMA",OBV_Zero_Cross_From_Below,RSI_Under_30,VWAP_Cross_From_Above,VWAP_Cross_From_Below
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-08-22 05:00:00+00:00,169.65,169.80,169.65,169.80,521,6,169.756526,169.800000,169.800000,169.800000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-08-22 06:00:00+00:00,169.22,169.22,169.22,169.22,100,1,169.220000,169.477778,169.495500,169.504200,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-08-22 06:45:00+00:00,169.20,169.20,169.20,169.20,299,1,169.200000,169.363934,169.386994,169.398718,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-08-22 07:15:00+00:00,169.10,169.10,169.10,169.10,114,6,169.101754,169.274526,169.304143,169.319499,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-08-22 08:00:00+00:00,170.29,170.29,169.01,169.29,69053,1471,169.268029,169.279129,169.300722,169.313118,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# Generate the trading signals 1 (entry) or -1 (exit) for a long position trading algorithm
# where -1 is when the Close price is less than the BB_LOWER window
# where 1 is when the Close price is greater the the BB_UPPER window
for index, row in signals_df.iterrows():
    if row["close"] < row["BB_LOWER"]:
        signals_df.loc[index, "Bollinger_Bands_Below_Lower_BB"] = 1
    if row["close"] > row["BB_UPPER"]:
        signals_df.loc[index,"Bollinger_Bands_Above_Upper_BB"] = 1

# Generate the trading signal 1 or 0
# where 1 is when the Short window is greater than (or crosses over) the Long Window
# where 0 is when the Short window is under the Long window
signals_df["9EMA/20EMA_Cross, 9EMA>20EMA"][9:] = np.where(
    signals_df["9EMA"][9:] > signals_df["20EMA"][9:], 1.0, 0.0)
# Calculate the points in time at which a position should be taken, 1 or -1, when there is a cross
signals_df["9EMA/20EMA_Cross, 9EMA>20EMA"] = signals_df["9EMA/20EMA_Cross, 9EMA>20EMA"].diff()
signals_df["9EMA/20EMA_Cross, 9EMA<20EMA"] = (signals_df["9EMA/20EMA_Cross, 9EMA>20EMA"]) * -1

# Generate the trading signal 1 or 0,
# where 1 is when the Short window is greater than (or crosses over) the Long Window
# where 0 is when the Short window is under the Long window
signals_df["50EMA/200SMA_Cross, 50EMA>200SMA"][50:] = np.where(
    signals_df["50EMA"][50:] > signals_df["200SMA"][50:], 1.0, 0.0)
# Calculate the points in time at which a position should be taken, 1 or -1, when the 50EMA Crosses the 200SMA
signals_df["50EMA/200SMA_Cross, 50EMA>200SMA"] = signals_df["50EMA/200SMA_Cross, 50EMA>200SMA"].diff()
signals_df["50EMA/200SMA_Cross, 50EMA<200SMA"] = (signals_df["50EMA/200SMA_Cross, 50EMA>200SMA"]) * -1


# WORK IN PROGRESS - WILL ADD SOON
# # Generate the trading signal 1 or 0,
# # where 1 is when the MACD is Increasing
# # where 0 is when the MACD is Decreasing
# n = signals_df["MACD_Rate"].index
# signals_df["MACD_Rate"] = np.where(
#     signals_df["MACD"][n+1] > signals_df["MACD"][n], 1.0, 0.0)
# # Calculate the points in time at which a position should be taken, 1 or -1
# signals_df["MACD_Rate"] = signals_df["MACD_Rate"].diff()

# Generate the trading signal 1 or 0,
# where 1 is when the OBV is Positive
# where 0 is when the OBV is Negative
signals_df["OBV_Zero_Cross_From_Above"] = np.where(
    signals_df["OBV"] > 0, 1.0, 0.0)
# Calculate the points in time at which a position should be taken, 1 or -1, when OBV crosses the zero-line
signals_df["OBV_Zero_Cross_From_Above"] = signals_df["OBV_Zero_Cross_From_Above"].diff()
signals_df["OBV_Zero_Cross_From_Below"] = (signals_df["OBV_Zero_Cross_From_Above"]) * -1

# signals_df["OBV_Zero_Cross_From_Above"].dropna()


# Generate the trading signals 1 (entry) or -1 (exit) for a long position trading algorithm
# where -1 is when the RSI is below 30
# where 1 is when the RSI is above 70
for index, row in signals_df.iterrows():
    if 30 > row["RSI"]:
        signals_df.loc[index, "RSI_Under_30"] = 1
    if 70 < row["RSI"]:
        signals_df.loc[index,"RSI_Over_70"] = 1

# Generate the trading signal 1 or 0,
# where 1 is when the price is above VWAP
# where 0 is when the price is below VWAP
signals_df["VWAP_Cross_From_Above"] = np.where(
    signals_df["vwap"] <= signals_df["close"], 1.0, 0)
# Calculate the points in time at which a position should be taken, 1 or -1, when price crosses VWAP
signals_df["VWAP_Cross_From_Above"] = signals_df["VWAP_Cross_From_Above"].diff()
signals_df["VWAP_Cross_From_Below"] = (signals_df["VWAP_Cross_From_Above"]) * -1


signals_df["VWAPDIFF"] = signals_df["VWAP_Cross_From_Above"] + signals_df["VWAP_Cross_From_Below"]


signals_df.head(100)

C:\Users\range\anaconda3\envs\dev\envs\dev\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\range\anaconda3\envs\dev\envs\dev\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,open,high,low,close,volume,trade_count,vwap,9EMA,20EMA,50EMA,...,OBV_Zero_Cross_From_Above,RSI_Over_70,Bollinger_Bands_Below_Lower_BB,"9EMA/20EMA_Cross, 9EMA<20EMA","50EMA/200SMA_Cross, 50EMA<200SMA",OBV_Zero_Cross_From_Below,RSI_Under_30,VWAP_Cross_From_Above,VWAP_Cross_From_Below,VWAPDIFF
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-08-22 05:00:00+00:00,169.6500,169.80,169.65,169.8000,521,6,169.756526,169.800000,169.800000,169.800000,...,NaN,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
2022-08-22 06:00:00+00:00,169.2200,169.22,169.22,169.2200,100,1,169.220000,169.477778,169.495500,169.504200,...,0.0,0.0,0.0,-0.0,-0.0,-0.0,1.0,0.0,-0.0,0.0
2022-08-22 06:45:00+00:00,169.2000,169.20,169.20,169.2000,299,1,169.200000,169.363934,169.386994,169.398718,...,0.0,0.0,0.0,-0.0,-0.0,-0.0,1.0,0.0,-0.0,0.0
2022-08-22 07:15:00+00:00,169.1000,169.10,169.10,169.1000,114,6,169.101754,169.274526,169.304143,169.319499,...,0.0,0.0,0.0,-0.0,-0.0,-0.0,1.0,-1.0,1.0,0.0
2022-08-22 08:00:00+00:00,170.2900,170.29,169.01,169.2900,69053,1471,169.268029,169.279129,169.300722,169.313118,...,1.0,0.0,0.0,-0.0,-0.0,-1.0,1.0,1.0,-1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-23 13:30:00+00:00,167.1200,168.14,167.08,167.1700,4630148,53976,167.657188,167.437427,167.604220,167.780299,...,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,0.0
2022-08-23 13:45:00+00:00,167.1800,168.41,167.14,168.0100,3556173,34771,167.998606,167.551942,167.642868,167.789497,...,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,1.0,-1.0,0.0
2022-08-23 14:00:00+00:00,168.0300,168.71,167.70,168.5100,2746376,28125,168.227356,167.743553,167.725457,167.818324,...,0.0,0.0,0.0,-1.0,-0.0,-0.0,0.0,0.0,-0.0,0.0


In [5]:
# here we create the exit column, our "y", for use in supervised ML
# How many rows are in the signals_df? for use in modifying DataFrame
periods = signals_df.shape[0]
# Exit is the labeled target for ML, Exit Price is for use in Pnl Metrics
signals_df["Exit"] = 0
signals_df["Exit Price"] = 0

# we also figure out our exit price
# target price signaling a win will be 1
# stop price signaling a loss will be -1
# loop thru the dataframe, from row 200 to the end (periods)
for j in range(200, periods):
    # entries will be on candle close
    entry = signals_df["close"].iloc[j]
    # calculate volatility for each candle
    atr = signals_df["ATR"].iloc[j]
    # stop is entry price minus the average volatility for the entry period
    stop = entry - atr
    # target is entry price plus the average volatility for the entry period times a multiplier
    target = entry + (3 * atr)
    # loop again thru the dataset to compare j entry price to future closing prices to see if we hit target or stop
    for k in range(j + 1, periods):
        # current low of the candle
        curr_low = signals_df["low"].iloc[k]
        # current high of the candle
        curr_high = signals_df["high"].iloc[k]
        # record and break if we hit stop or target, if not we check the next k period
        # if current low breaks our stop we should've sold: -1 in our "Exit" column
        if curr_low <= stop:
            signals_df["Exit Price"].iloc[j] = stop
            signals_df["Exit"].iloc[j] = -1
            # if we hit the stop break the inner loop to check the next row
            break
        # if current high breaks our target we should've sold: +1 in our "Exit" column
        elif curr_high >= target:
            signals_df["Exit Price"].iloc[j] = target
            signals_df["Exit"].iloc[j] = 1
            # if we hit the target break the inner loop to check the next row
            break

# drop beginning columns to avoid NaN values from EMA/SMA calculations
signals_df = signals_df[200:]

signals_df

C:\Users\range\anaconda3\envs\dev\envs\dev\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,open,high,low,close,volume,trade_count,vwap,9EMA,20EMA,50EMA,...,Bollinger_Bands_Below_Lower_BB,"9EMA/20EMA_Cross, 9EMA<20EMA","50EMA/200SMA_Cross, 50EMA<200SMA",OBV_Zero_Cross_From_Below,RSI_Under_30,VWAP_Cross_From_Above,VWAP_Cross_From_Below,VWAPDIFF,Exit,Exit Price
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-08-24 23:15:00+00:00,167.9400,167.9400,167.83,167.83,7098,113,167.877556,167.865654,167.783150,167.635390,...,0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,0.0,1,168.564936
2022-08-24 23:30:00+00:00,167.9199,167.9199,167.82,167.85,5653,73,167.844483,167.862523,167.789517,167.643809,...,0.0,-0.0,-0.0,-0.0,0.0,1.0,-1.0,0.0,1,168.518486
2022-08-24 23:45:00+00:00,167.8300,167.9300,167.70,167.84,23294,269,167.839347,167.858019,167.794325,167.651505,...,0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,0.0,1,168.484914
2022-08-25 04:30:00+00:00,167.8300,167.8300,167.83,167.83,270,9,167.830000,167.852415,167.797722,167.658507,...,0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,0.0,1,168.354979
2022-08-25 06:15:00+00:00,168.4000,168.4000,168.40,168.40,192,11,168.493333,167.961932,167.855082,167.687593,...,0.0,-0.0,-0.0,-0.0,0.0,-1.0,1.0,0.0,1,168.946407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-14 22:45:00+00:00,138.1400,138.1400,138.04,138.07,11808,274,138.074685,138.251212,138.596922,139.568290,...,0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,0.0,0,0.000000
2022-10-14 23:00:00+00:00,138.0700,138.0700,138.00,138.04,9133,379,138.016969,138.208970,138.543881,139.508357,...,0.0,-0.0,-0.0,-0.0,0.0,1.0,-1.0,0.0,0,0.000000
2022-10-14 23:15:00+00:00,138.0400,138.0800,138.02,138.05,8187,203,138.043907,138.177176,138.496845,139.451167,...,0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,0.0,0,0.000000


In [6]:
# check if there is potentially-unwanted zeros in the dataframe
signals_df["Exit"].value_counts()

-1    1716
 1     656
 0      19
Name: Exit, dtype: int64

In [7]:
# Remove all unwanted zeros from the exit column
signals_df = signals_df.loc[signals_df["Exit"] != 0]
signals_df["Exit"].value_counts()

-1    1716
 1     656
Name: Exit, dtype: int64

In [8]:
# Create a new dataframe to hold time-periods where a signal occured
column_names = signals_df.columns
df = pd.DataFrame(columns = column_names)

# If a bollinger-bands signal occurred, add that row to our dataframe
for index, row in signals_df.iterrows(): 
    if signals_df["Bollinger_Bands_Above_Upper_BB"].loc[index] == 1:
        df.loc[index] = signals_df.loc[index]
    if signals_df["Bollinger_Bands_Below_Lower_BB"].loc[index] == 1:
        df.loc[index] = signals_df.loc[index]
        
# If a 9EMA/20EMA cross signal occurred, add that row to our dataframe
for index, row in signals_df.iterrows(): 
    if signals_df["9EMA/20EMA_Cross, 9EMA>20EMA"].loc[index] == 1:
        df.loc[index] = signals_df.loc[index]
    if signals_df["9EMA/20EMA_Cross, 9EMA<20EMA"].loc[index] == 1:
        df.loc[index] = signals_df.loc[index]
        
# If a 50EMA/200SMA cross signal occurred, add that row to our dataframe
for index, row in signals_df.iterrows(): 
    if signals_df["50EMA/200SMA_Cross, 50EMA>200SMA"].loc[index] == 1:
        df.loc[index] = signals_df.loc[index]
    if signals_df["50EMA/200SMA_Cross, 50EMA<200SMA"].loc[index] == 1:
        df.loc[index] = signals_df.loc[index]
        
# If an OBV_Zero_Cross signal occurred, add that row to our dataframe
for index, row in signals_df.iterrows(): 
    if signals_df["OBV_Zero_Cross_From_Above"].loc[index] == 1:
        df.loc[index] = signals_df.loc[index]
    if signals_df["OBV_Zero_Cross_From_Below"].loc[index] == 1:
        df.loc[index] = signals_df.loc[index]
        
# If a RSI_Position signal occurred, add that row to our dataframe
for index, row in signals_df.iterrows(): 
    if signals_df["RSI_Over_70"].loc[index] == 1:
        df.loc[index] = signals_df.loc[index]
    if signals_df["RSI_Under_30"].loc[index] == 1:
        df.loc[index] = signals_df.loc[index]
        
# If a VWAP_Cross signal occurred, add that row to our dataframe
for index, row in signals_df.iterrows(): 
    if signals_df["VWAP_Cross_From_Above"].loc[index] == 1:
        df.loc[index] = signals_df.loc[index]
    if signals_df["VWAP_Cross_From_Below"].loc[index] == 1:
        df.loc[index] = signals_df.loc[index]
        
df

,open,high,low,close,volume,trade_count,vwap,9EMA,20EMA,50EMA,...,Bollinger_Bands_Below_Lower_BB,"9EMA/20EMA_Cross, 9EMA<20EMA","50EMA/200SMA_Cross, 50EMA<200SMA",OBV_Zero_Cross_From_Below,RSI_Under_30,VWAP_Cross_From_Above,VWAP_Cross_From_Below,VWAPDIFF,Exit,Exit Price
2022-08-25 06:15:00+00:00,168.40,168.40,168.4000,168.400,192.0,11.0,168.493333,167.961932,167.855082,167.687593,...,0.0,-0.0,-0.0,-0.0,0.0,-1.0,1.0,0.0,1.0,168.946407
2022-08-25 06:45:00+00:00,169.99,169.99,169.2000,169.200,558.0,9.0,169.628298,168.209546,167.983169,167.746919,...,0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,0.0,-1.0,168.919293
2022-08-25 08:00:00+00:00,169.02,169.80,168.6100,169.380,29432.0,679.0,169.375631,168.443636,168.116201,167.810977,...,0.0,-0.0,-1.0,-0.0,0.0,1.0,-1.0,0.0,-1.0,169.022864
2022-08-25 08:15:00+00:00,169.37,169.39,169.2400,169.240,7434.0,209.0,169.288499,168.602909,168.223229,167.867031,...,0.0,-0.0,-0.0,-0.0,0.0,-1.0,1.0,0.0,-1.0,168.877150
2022-08-25 08:30:00+00:00,169.20,169.40,169.2000,169.400,7243.0,201.0,169.339739,168.762327,168.335303,167.927162,...,0.0,-0.0,-0.0,-0.0,0.0,1.0,-1.0,0.0,-1.0,169.028579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-14 18:15:00+00:00,138.88,138.95,138.5000,138.740,2000193.0,17716.0,138.646219,139.277279,139.966866,140.814994,...,0.0,-0.0,-0.0,-0.0,0.0,1.0,-1.0,0.0,-1.0,138.089764
2022-10-14 18:30:00+00:00,138.73,138.86,138.4501,138.645,2217155.0,17348.0,138.692574,139.150823,139.840974,140.729896,...,0.0,-0.0,-0.0,-0.0,0.0,-1.0,1.0,0.0,-1.0,138.042986
2022-10-14 18:45:00+00:00,138.63,138.82,138.3900,138.805,1839329.0,15988.0,138.619807,139.081658,139.742310,140.654410,...,0.0,-0.0,-0.0,-0.0,0.0,1.0,-1.0,0.0,-1.0,138.234414
2022-10-14 19:15:00+00:00,138.43,138.90,138.3100,138.825,2621917.0,21444.0,138.652907,138.926077,139.541876,140.498861,...,0.0,-0.0,-0.0,-0.0,0.0,1.0,-1.0,0.0,-1.0,138.252986


In [9]:
# choose if you want continuous or discrete features
continuous_features = ["volume", "trade_count", "vwap", "9EMA", "20EMA", "50EMA", "200SMA", "ATR", "RSI", "BB_UPPER", "BB_MIDDLE", "BB_LOWER", "MACD"]
discrete_features = ["Bollinger_Bands_Above_Upper_BB", "Bollinger_Bands_Below_Lower_BB", "9EMA/20EMA_Cross, 9EMA>20EMA", "9EMA/20EMA_Cross, 9EMA<20EMA", "50EMA/200SMA_Cross, 50EMA>200SMA", "50EMA/200SMA_Cross, 50EMA<200SMA", "RSI_Over_70", "RSI_Under_30", "VWAP_Cross_From_Above", "VWAP_Cross_From_Below"]
# features = ["volume", "trade_count", "vwap", "9EMA", "20EMA", "50EMA", "200SMA", "ATR", "RSI", "BB_UPPER", "BB_MIDDLE", "BB_LOWER", "MACD","Bollinger_Bands_Position", "9EMA/20EMA_Cross", "50EMA/200SMA_Cross, 50EMA<200SMA", "OBV_Zero_Cross_From_Above", "ROBV_Zero_Cross_From_Below", "VWAP_Cross"]

X = df[discrete_features]

# 1 means a buy would've produced a profit, -1 means a sale would've produced a profit
y = df["Exit"]
# X
y

2022-08-25 06:15:00+00:00    1.0
2022-08-25 06:45:00+00:00   -1.0
2022-08-25 08:00:00+00:00   -1.0
2022-08-25 08:15:00+00:00   -1.0
2022-08-25 08:30:00+00:00   -1.0
                            ... 
2022-10-14 18:15:00+00:00   -1.0
2022-10-14 18:30:00+00:00   -1.0
2022-10-14 18:45:00+00:00   -1.0
2022-10-14 19:15:00+00:00   -1.0
2022-10-14 20:00:00+00:00   -1.0
Name: Exit, Length: 1555, dtype: float64

In [10]:
# check for potentially unwanted zeros
df["Exit"].value_counts()

-1.0    1120
 1.0     435
Name: Exit, dtype: int64

In [11]:
for i in discrete_features:
    df[i] = df[i].replace(-1, 0)

In [12]:
# Note: check if we have duplicate rows in the dataframe
df.isnull().sum()

open                                 0
high                                 0
low                                  0
close                                0
volume                               0
trade_count                          0
vwap                                 0
9EMA                                 0
20EMA                                0
50EMA                                0
200SMA                               0
ATR                                  0
OBV                                 23
RSI                                  0
BB_UPPER                             0
BB_MIDDLE                            0
BB_LOWER                             0
MACD                                 0
Bollinger_Bands_Above_Upper_BB       0
9EMA/20EMA_Cross, 9EMA>20EMA         0
50EMA/200SMA_Cross, 50EMA>200SMA     0
OBV_Zero_Cross_From_Above            0
RSI_Over_70                          0
Bollinger_Bands_Below_Lower_BB       0
9EMA/20EMA_Cross, 9EMA<20EMA         0
50EMA/200SMA_Cross, 50EMA

In [13]:
for i in discrete_features:
    print(df[i].value_counts())

0.0    1415
1.0     140
Name: Bollinger_Bands_Above_Upper_BB, dtype: int64
0.0    1411
1.0     144
Name: Bollinger_Bands_Below_Lower_BB, dtype: int64
0.0    1500
1.0      55
Name: 9EMA/20EMA_Cross, 9EMA>20EMA, dtype: int64
-0.0    1499
 1.0      56
Name: 9EMA/20EMA_Cross, 9EMA<20EMA, dtype: int64
0.0    1547
1.0       8
Name: 50EMA/200SMA_Cross, 50EMA>200SMA, dtype: int64
-0.0    1548
 1.0       7
Name: 50EMA/200SMA_Cross, 50EMA<200SMA, dtype: int64
0.0    1408
1.0     147
Name: RSI_Over_70, dtype: int64
0.0    1325
1.0     230
Name: RSI_Under_30, dtype: int64
0.0    952
1.0    603
Name: VWAP_Cross_From_Above, dtype: int64
-0.0    957
 1.0    598
Name: VWAP_Cross_From_Below, dtype: int64


In [15]:
for i in discrete_features:
    df[i] = df[i].replace(-0, 0)
    print(df[i].value_counts())

0.0    1415
1.0     140
Name: Bollinger_Bands_Above_Upper_BB, dtype: int64
0.0    1411
1.0     144
Name: Bollinger_Bands_Below_Lower_BB, dtype: int64
0.0    1500
1.0      55
Name: 9EMA/20EMA_Cross, 9EMA>20EMA, dtype: int64
0.0    1499
1.0      56
Name: 9EMA/20EMA_Cross, 9EMA<20EMA, dtype: int64
0.0    1547
1.0       8
Name: 50EMA/200SMA_Cross, 50EMA>200SMA, dtype: int64
0.0    1548
1.0       7
Name: 50EMA/200SMA_Cross, 50EMA<200SMA, dtype: int64
0.0    1408
1.0     147
Name: RSI_Over_70, dtype: int64
0.0    1325
1.0     230
Name: RSI_Under_30, dtype: int64
0.0    952
1.0    603
Name: VWAP_Cross_From_Above, dtype: int64
0.0    957
1.0    598
Name: VWAP_Cross_From_Below, dtype: int64
